In [14]:
import pandas as pd

project_path = '/home/wjunneng/Ubuntu/2019-CCF-Prediction-Of-The-Quality-Compliance-Rate-Of-Workpieces-In-Discrete-Manufacturing-Processes'

first_round_testing_data_path = project_path + '/data/original/first_round_testing_data.csv'
first_round_training_data_path = project_path + '/data/original/first_round_training_data.csv'

first_round_testing_data = pd.read_csv(first_round_testing_data_path)
first_round_training_data = pd.read_csv(first_round_training_data_path)

select_columns = ['Parameter1', 'Parameter2', 'Parameter3',
       'Parameter4', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8',
       'Parameter9', 'Parameter10']

print(first_round_testing_data.columns)
print(first_round_training_data.columns)


Index(['Group', 'Parameter1', 'Parameter10', 'Parameter2', 'Parameter3',
       'Parameter4', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8',
       'Parameter9'],
      dtype='object')
Index(['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4', 'Parameter5',
       'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10',
       'Attribute1', 'Attribute2', 'Attribute3', 'Attribute4', 'Attribute5',
       'Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Attribute10',
       'Quality_label'],
      dtype='object')


In [15]:
data = pd.concat([first_round_testing_data[select_columns], first_round_training_data[select_columns]], axis=0, ignore_index=True)

data.head()


,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10
0,0.235279,0.351488,0.186814,0.086456,1.434060,1.230409,0.038483,2.931083,2.005852,0.195680
1,0.043799,32.590688,0.886285,29.702490,0.027434,0.016852,0.600827,0.118077,887.611257,0.523976
2,21.466737,11.847792,79499.554454,13544.605343,0.027434,0.016852,0.600827,0.118077,887.611257,0.523976
3,0.703795,0.018080,317.015599,38.087562,0.428092,0.314162,0.038483,17.850021,0.051850,0.010192
4,10.447960,15.983891,0.996049,16.715690,0.477823,0.464037,0.038483,17.850021,0.051850,0.010192


In [16]:
data['Parameter5'] = data['Parameter5'].apply(lambda x: int(round(x)))
print(data['Parameter5'].value_counts())


0     2565
1     2044
2     1423
3      975
7      931
5      755
4      668
9      357
8      290
28     230
6      213
16     212
12     199
13     181
14     144
35     122
31     114
18     112
10     108
20     106
25      88
22      85
43      35
39      32
48       3
60       3
84       3
67       1
75       1
Name: Parameter5, dtype: int64


In [17]:
data['Parameter6'] = data['Parameter6'].apply(lambda x: int(round(x)))
print(data['Parameter6'].value_counts())


0     3051
1     1563
3     1124
2     1079
11     834
4      669
6      668
5      517
7      504
9      362
16     344
19     294
13     294
23     277
28     262
34     120
41      38
Name: Parameter6, dtype: int64


In [18]:
data['Parameter7'] = data['Parameter7'].apply(lambda x: int(round(x)))
print(data['Parameter7'].value_counts())


0        5166
1        3152
2        1761
146       733
2287      655
37        186
579       154
1150       88
9          70
35699      29
9035        6
Name: Parameter7, dtype: int64


In [19]:
data['Parameter8'] = data['Parameter8'].apply(lambda x: int(round(x)))
print(data['Parameter8'].value_counts())


0        5527
18       3508
15       1409
3         834
7         334
22        130
1209       96
162        50
10         44
73         43
4           9
542         4
66985       3
1           3
20086       3
8999        1
363         1
243         1
Name: Parameter8, dtype: int64


In [20]:
data['Parameter9'] = data['Parameter9'].apply(lambda x: int(round(x)))
print(data['Parameter9'].value_counts())


1            5566
0            2481
7            2338
888           613
78            341
262           313
2             224
23            101
10153           6
173808298       5
1328408         4
4887            3
16530           3
116135          1
3002            1
Name: Parameter9, dtype: int64


In [21]:
data['Parameter10'] = data['Parameter10'].apply(lambda x: int(round(x)))
print(data['Parameter10'].value_counts())


0      5328
1      1456
14      503
3       441
7       407
37      405
4       374
27      372
10      367
5       350
517     300
19      283
2       279
52      205
72      191
139     190
100     179
193     147
268     116
372     107
Name: Parameter10, dtype: int64


In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold
from itertools import product


class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode

        :param n_splits: the number of splits used in mean encoding

        :param target_type: str, 'regression' or 'classification'

        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()

        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()

        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)

        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target,
                        self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None,
                        self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new


In [25]:
print(data.shape[0])
print(first_round_training_data.shape[0])
print(first_round_testing_data.shape[0])


first_round_training_data['Quality_label'] = first_round_training_data['Quality_label'].map(dict(zip(first_round_training_data['Quality_label'].unique(), range(first_round_training_data['Quality_label'].nunique()))))
y = first_round_training_data['Quality_label']

X = data.loc[:first_round_training_data.shape[0],:]
X_test = data.loc[first_round_training_data.shape[0]:,]

print(X.head())
print(X_test.head())
print(y.value_counts())


12000
6000
6000
   Parameter1  Parameter2    Parameter3    Parameter4  Parameter5  Parameter6  \
0    0.235279    0.351488      0.186814      0.086456           1           1   
1    0.043799   32.590688      0.886285     29.702490           0           0   
2   21.466737   11.847792  79499.554454  13544.605343           0           0   
3    0.703795    0.018080    317.015599     38.087562           0           0   
4   10.447960   15.983891      0.996049     16.715690           0           0   

   Parameter7  Parameter8  Parameter9  Parameter10  
0           0           3           2            0  
1           1           0         888            1  
2           1           0         888            1  
3           0          18           0            0  
4           0          18           0            0  
      Parameter1  Parameter2  Parameter3  Parameter4  Parameter5  Parameter6  \
6000    0.001660    0.591013  147.608373   38.186345           0           0   
6001    1.601749   

In [26]:
columns = ['Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']
#声明需要平均数编码的特征
MeanEnocodeFeature = columns
#声明平均数编码的类
ME = MeanEncoder(MeanEnocodeFeature)
#对训练数据集的X和y进行拟合
X = ME.fit_transform(X,y)
#对测试集进行编码
X_test = ME.transform(X_test)

for column in columns:
    print(X[column].value_counts())
    print(X_test[column].value_counts())



/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*in

0     1232
1      966
2      734
3      541
7      456
5      411
4      367
9      196
8      165
16     123
6      120
12     110
13     103
14      85
20      73
28      68
18      65
10      53
35      36
31      33
22      23
25      22
43       9
39       3
60       2
84       2
48       1
67       1
75       1
Name: Parameter5, dtype: int64
0     1334
1     1078
2      689
7      475
3      434
5      344
4      301
28     162
9      161
8      125
6       93
12      89
16      89
35      86
31      81
13      78
25      66
22      62
14      59
10      55
18      47
20      33
39      29
43      26
48       2
84       1
60       1
Name: Parameter5, dtype: int64
0     1458
1      796
3      572
2      514
11     410
6      372
4      360
7      281
5      253
16     196
9      183
19     177
13     137
28     134
23     120
34      34
41       4
Name: Parameter6, dtype: int64
0     1594
1      767
2      565
3      552
11     424
4      309
6      296
5      264
7      223
9    

In [13]:
for column in ['Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']:
    #声明需要平均数编码的特征
    MeanEnocodeFeature = [column] 
    #声明平均数编码的类
    ME = MeanEncoder(MeanEnocodeFeature)
    #对训练数据集的X和y进行拟合
    X = ME.fit_transform(X,y)
    #对测试集进行编码
    X_test = ME.transform(X_test)
    
    print(X[column].value_counts())
    print(X_test[column].value_counts())


/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*in

0     1232
1      966
2      734
3      541
7      456
5      411
4      367
9      196
8      165
16     123
6      120
12     110
13     103
14      85
20      73
28      68
18      65
10      53
35      36
31      33
22      23
25      22
43       9
39       3
60       2
84       2
48       1
67       1
75       1
Name: Parameter5, dtype: int64
0     1334
1     1078
2      689
7      475
3      434
5      344
4      301
28     162
9      161
8      125
6       93
12      89
16      89
35      86
31      81
13      78
25      66
22      62
14      59
10      55
18      47
20      33
39      29
43      26
48       2
84       1
60       1
Name: Parameter5, dtype: int64
0     1458
1      796
3      572
2      514
11     410
6      372
4      360
7      281
5      253
16     196
9      183
19     177
13     137
28     134
23     120
34      34
41       4
Name: Parameter6, dtype: int64
0     1594
1      767
2      565
3      552
11     424
4      309
6      296
5      264
7      223
9    